<a href="https://colab.research.google.com/github/tofighi/MachineLearning/blob/master/datasets/LLM/OpenAssistant/H2O_LLM_Studio_CLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune a large language model using [H2O LLM Studio](https://github.com/h2oai/h2o-llmstudio)

In this notebook, we demonstrate how one can finetune a large language model easily using the **CLI interface** of H2O LLM Studio.

In [1]:
!git clone https://github.com/h2oai/h2o-llmstudio.git
!cd h2o-llmstudio && git checkout ce10af57ff118a2bbb81b5b3eae12273e290299a -q
!cp -r h2o-llmstudio/. ./
!rm -r h2o-llmstudio

Cloning into 'h2o-llmstudio'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (315/315), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 315 (delta 179), reused 273 (delta 138), pack-reused 0
Receiving objects: 100% (315/315), 10.55 MiB | 16.54 MiB/s, done.
Resolving deltas: 100% (179/179), done.


In [2]:
# Install pyhon 3.10 that will be used within pipenv
!sudo add-apt-repository ppa:deadsnakes/ppa -y > /dev/null
!sudo apt install python3.10 python3.10-distutils psmisc -y > /dev/null
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10 > /dev/null
    
# install requirements
!make setup > /dev/null



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 6.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Creating a virtualenv for this project...
Pipfile: /content/Pipfile
Using /usr/bin/python3.10 (3.10.11) to create virtualenv...
⠹ Creating virtual environment...created virtual environment CPython3.10.11.final.0-64 in 874ms
  creator Venv(dest=/root/.local/share/virtualenvs/content-cQIIIOO2, clear=False, no_vcs_ignore=False, global=False, describe=CPython3Posix)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.1, setuptools==

### OASST Data

You can get the OASST dataset from [Kaggle](https://www.kaggle.com/code/philippsinger/openassistant-conversations-dataset-oasst1), or prepare it yourself as shown next.

In [3]:
!python -m pip install datasets > /dev/null
!mkdir data
!mkdir data/oasst-data

In [4]:
import numpy as np
import pandas as pd 
pd.options.mode.chained_assignment = None
from datasets import load_dataset

ds = load_dataset("OpenAssistant/oasst1")
train = ds['train']
val = ds['validation']

train = pd.DataFrame(train)
val = pd.DataFrame(val)

def prep_data(df):
    df_assistant = df[(df.role=="assistant") & (df["rank"]==0.0)]
    df_prompter = df[(df.role=="prompter")]
    df_prompter = df_prompter.set_index("message_id")
    df_assistant["output"] = df_assistant["text"].values
    
    inputs = []
    for idx, row in df_assistant.iterrows():
        input = df_prompter.loc[row.parent_id]
        inputs.append(input.text)
        
    df_assistant["instruction"] = inputs
    
    df_assistant = df_assistant[df_assistant.lang=="en"]
    
    df_assistant = df_assistant[["instruction", "output"]]
    
    return df_assistant

df_train = prep_data(train)
df_val = prep_data(val)

pd.concat([df_train, df_val]).reset_index(drop=True).to_csv("data/oasst-data/train_full.csv", index=False)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/4401 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/84437 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/OpenAssistant___parquet/OpenAssistant--oasst1-ea605663b798f601/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

## Configurations

In H2O LLM Studio, we use dataclasses to specify various [finetuning parameters](https://github.com/h2oai/h2o-llmstudio/blob/main/docs/parameters.md).

In [5]:
%%writefile cfg_notebook.py

import os
from dataclasses import dataclass

from llm_studio.python_configs.text_causal_language_modeling_config import ConfigProblemBase, ConfigNLPCausalLMDataset, \
    ConfigNLPCausalLMTokenizer, ConfigNLPAugmentation, ConfigNLPCausalLMArchitecture, ConfigNLPCausalLMTraining, \
    ConfigNLPCausalLMPrediction, ConfigNLPCausalLMEnvironment, ConfigNLPCausalLMLogging


ROOT_DIR = "./data/oasst-data/"

@dataclass
class Config(ConfigProblemBase):
    output_directory: str = "output/demo_oasst-data/"
    experiment_name: str = "demo_experiment"
    llm_backbone: str = "EleutherAI/pythia-1.4b-deduped"

    dataset: ConfigNLPCausalLMDataset = ConfigNLPCausalLMDataset(
        train_dataframe=os.path.join(ROOT_DIR, "train_full.csv"),
        
        validation_strategy="automatic",
        validation_dataframe="",
        validation_size=0.01,

        prompt_column=("instruction",),
        answer_column="output",
        text_prompt_start="",
        text_answer_separator="",

        add_eos_token_to_prompt=True,
        add_eos_token_to_answer=True,
        mask_prompt_labels=False,

    )
    tokenizer: ConfigNLPCausalLMTokenizer = ConfigNLPCausalLMTokenizer(
        max_length_prompt=128,
        max_length_answer=128,
        max_length=256,
        padding_quantile=1.0
    )
    augmentation: ConfigNLPAugmentation = ConfigNLPAugmentation(token_mask_probability=0.0)
    architecture: ConfigNLPCausalLMArchitecture = ConfigNLPCausalLMArchitecture(
        backbone_dtype="float16",
        gradient_checkpointing=False,
        force_embedding_gradients=False,
        intermediate_dropout=0
    )
    training: ConfigNLPCausalLMTraining = ConfigNLPCausalLMTraining(
        loss_function="CrossEntropy",
        optimizer="AdamW",

        learning_rate=0.00015,

        batch_size=4,
        drop_last_batch=True,
        epochs=1,
        schedule="Cosine",
        warmup_epochs=0.0,

        weight_decay=0.0,
        gradient_clip=0.0,
        grad_accumulation=1,

        lora=True,
        lora_r=4,
        lora_alpha=16,
        lora_dropout=0.05,
        lora_target_modules="",

        save_best_checkpoint=False,
        evaluation_epochs=1.0,
        evaluate_before_training=False,
    )
    prediction: ConfigNLPCausalLMPrediction = ConfigNLPCausalLMPrediction(
        metric="BLEU",

        min_length_inference=2,
        max_length_inference=256,
        batch_size_inference=0,

        do_sample=False,
        num_beams=2,
        temperature=0.3,
        repetition_penalty=1.2,
    )
    environment: ConfigNLPCausalLMEnvironment = ConfigNLPCausalLMEnvironment(
        mixed_precision=True,
        number_of_workers=4,
        seed=1
    )

Writing cfg_notebook.py


In [6]:
%%writefile run.sh

pipenv run python train.py -C cfg_notebook.py & 

wait
echo "all done"

Writing run.sh


In [7]:
!sh run.sh

/content/train.py:15: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils import util

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/root/.local/share/virtualenvs/content-cQIIIOO2/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/root/.local/share/virtualenvs/content-cQIIIOO2/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//ipykernel.pylab.backend_inline'), PosixPath('module')}
  warn(msg)
/root/.local/share/virtualenvs/content-cQIIIOO2/lib/python3.10/si

In [8]:
import pandas as pd
val_outputs = pd.read_csv("output/demo_oasst-data/validation_predictions.csv")

In [9]:
val_outputs.head()

,instruction,output,pred_output
0,What types of tests do we have in software dev...,There are many types of tests in software deve...,There are many types of tests in software deve...
1,Can you make it about 50% shorter and more exc...,You are looking for a design? We‘ve got you co...,Sure! Here's a 50% shorter and more exciting v...
2,write a story,"Once upon a time, in a kingdom far away, there...","I'm sorry, but I don't know how to write a sto..."
3,I'm currently on the phone with a customer sup...,State that you are a loyal customer to them fo...,"Hello,\nI'm sorry to hear that you're experien..."
4,"If we're going to war, I'm in the demographic ...",It is difficult to predict the likelihood of i...,It is unlikely that you will be drafted in the...


In [10]:
for _, row in val_outputs.iloc[41:42].iterrows():
    print("============")
    print()
    print(row.instruction)
    print()
    print("-----Target Answer-----")
    print()
    print(row.output)
    print()
    print("-----Predicted Answer-----")
    print()
    print(row.pred_output)
    print()


What are the advantages of H.265 encoding over H.264?

-----Target Answer-----

H.265, also known as High Efficiency Video Coding (HEVC), is an advanced video compression standard that offers several advantages over its predecessor H.264 (AVC). It provides improved compression efficiency and video quality, enabling smaller file sizes and better playback performance over limited bandwith connections, particularly for high-resolution and HDR video content.

In detail:

1.) Higher compression efficiency: H.265 can compress video files to half the size of H.264 while maintaining the same video quality, allowing for smaller file sizes and lower bandwidth usage. The biggest contributing factors for the higher compression efficiency is the use of coding tree units instead of macroblocks and improved motion compensation and spatial prediction.

2.) Improved image quality: H.265 uses advanced compression techniques that result in improved image quality while producing smaller file sizes, parti

### Inference and prompting

You can also load the trained model and manually prompt it.

In [12]:
!pipenv run python prompt.py --e output/demo_oasst-data/


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/root/.local/share/virtualenvs/content-cQIIIOO2/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/root/.local/share/virtualenvs/content-cQIIIOO2/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/root/.local/share/virtualenvs/content-cQIIIOO2/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), Posix